In [8]:
import tensorflow as tf
import pandas as pd
import sagemaker
tf.__version__

'2.1.3'

In [9]:
#Reading in data in folder
df = pd.read_csv('Iris.csv')
df = df.drop(['Id'], axis=1)
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [10]:
len(df)

150

In [11]:
#Splitting data in 80-20 split to use testing data for model inference later
train = df.iloc[:120,:]
test = df.iloc[121:,:]

In [12]:
train.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [13]:
#Train and test csv
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [14]:
#Create a sagemaker session to be able to upload data to s3
import boto3
sagemaker_session = sagemaker.Session()

In [15]:
bucket_name  = sagemaker_session.default_bucket()

In [16]:
#Uploading data to S3 bucket titled "tf-iris-data"
prefix = "data/tf-iris-data"
training_input_path = sagemaker_session.upload_data('train.csv', bucket=bucket_name, key_prefix=prefix + '/training')

In [18]:
#ensure training data has uploaded properly
training_data = pd.read_csv(training_input_path, sep = ',')
training_data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [19]:
test.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
121,5.6,2.8,4.9,2.0,Iris-virginica
122,7.7,2.8,6.7,2.0,Iris-virginica
123,6.3,2.7,4.9,1.8,Iris-virginica
124,6.7,3.3,5.7,2.1,Iris-virginica
125,7.2,3.2,6.0,1.8,Iris-virginica


In [20]:
#Need tf version and role for the Tensorflow Estimator
tf_version = tf.__version__
tf_version

'2.1.3'

In [21]:
#Sagemaker role, make sure you've allowed access to any S3 bucket
role = sagemaker.get_execution_role()

'arn:aws:iam::752783179453:role/service-role/AmazonSageMaker-ExecutionRole-20210921T073733'

In [22]:
#Use a tensorflow estimator from sagemaker to train model
from sagemaker.tensorflow import TensorFlow

tagKey="Project"
tagValue="Project_Iris_TF"

tf_estimator = TensorFlow(entry_point='train.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m4.xlarge',
                          framework_version=tf_version, 
                          py_version='py3',
                          script_mode=True,
                          tags=[{"Key": tagKey, "Value": tagValue}],
                          hyperparameters={
                              'epochs': 30
                          }
                         )

In [23]:
#Training
tf_estimator.fit({'train': training_input_path})

2022-12-14 11:58:43 Starting - Starting the training job...
2022-12-14 11:59:07 Starting - Preparing the instances for trainingProfilerReport-1671019123: InProgress
............
2022-12-14 12:01:07 Downloading - Downloading input data...
2022-12-14 12:01:27 Training - Training image download completed. Training in progress...2022-12-14 12:01:44,480 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2022-12-14 12:01:44,488 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-14 12:01:44,725 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-14 12:01:44,746 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-14 12:01:44,768 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-14 12:01:44,782 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_par